In [ ]:
def check_brackets(s):
    stack = []
    for ind, ch in enumerate(s, start=1):
        if ch in ('(', '[', '{'):
            stack.append((ch, ind))
        elif ch in (')', ']', '}'):
            if not stack:
                return ind
            top, _ = stack.pop()
            if (top == '(' and ch != ')') or \
                    (top == '[' and ch != ']') or \
                    (top == '{' and ch != '}'):
                return ind
    if stack:
        return stack[0][1]
    else:
        return 'Success'


def main():
    s = input()
    print(check_brackets(s))


def test():
    for inpt, outpt in [('([](){([])})', 'Success'), ('()[]}', 5), ('{{[()]]', 7)]:
        assert check_brackets(inpt) == outpt

if __name__ == "__main__":
    main()

In [ ]:
import sys


def Height_iterative(tree, n):
    heights = {}
    for child in range(n):
        parent = tree[child]
        heights[child] = 1
        while parent != -1:
            if parent in heights:
                heights[child] += heights[parent]
                break
            heights[child] += 1
            parent = tree[parent]

    return max(heights.values())


def Height_recursive(tree, root):
    height = 1
    for child in tree[root]:
        height = max(height, 1 + Height_recursive(tree, child))
    return height


def main():
    reader = (map(int, line.split()) for line in sys.stdin)
    [n], tree = next(reader), list(next(reader))
    # итеративный алгоритм
    print(Height_iterative(tree, n))

    # рекурсивный алгоритм
    tree_map = {i: set() for i in range(-1, n)}
    for child, parent in enumerate(tree):
        tree_map[parent].add(child)
    [root] = tree_map[-1]
    print(Height_recursive(tree_map, root))


def test():
    assert Height([4, -1, 4, 1, 1], 5) == 3
    assert Height([-1, 0, 4, 0, 3], 5) == 4
    assert Height([9, 7, 5, 5, 2, 9, 9, 9, 2, -1], 10) == 4

if __name__ == "__main__":
    sys.setrecursionlimit(20000)
    main()

In [ ]:
import sys
from collections import deque


class Buffer:
    def __init__(self, size):
        self.size = size
        self.buffer = deque([])  # храним время завершения обработки пакета
        self.log = []            # сюда пишем время начала обработки пакета
        self.__proc = 0          # время пока процессор будет занят

    def __check(self, arrival):
        if len(self.buffer) < self.size:
            return True
        elif arrival >= self.buffer[0]:
            self.buffer.popleft()
            return True
        return False

    def __push(self, arrival, duration):
        if self.buffer:
            start_proc = max(arrival, self.buffer[-1])
        else:
            start_proc = max(arrival, self.__proc)
            self.__proc = duration
        self.log.append(start_proc)
        self.buffer.append(start_proc + duration)

    def add_packet(self, arrival, duration):
        if self.__check(arrival):
            self.__push(arrival, duration)
        else:
            self.log.append(-1)


def main():
    reader = (tuple(map(int, line.split())) for line in sys.stdin)
    size, n  = next(reader)
    packets = list(reader)
    buffer = Buffer(size)
    for arrival, duration in packets:
        buffer.add_packet(arrival, duration)

    print(*buffer.log, sep='\n')


if __name__ == "__main__":
    main()

In [ ]:
import sys


class Stack:
    def __init__(self):
        self.stack = []

    def push(self, i):
        self.stack.append(i)

    def head(self):
        return self.stack[-1]

    def is_empty(self):
        if self.stack:
            return True
        return False


def main():
    reader = (line.split() for line in sys.stdin)
    [q] = next(reader)
    stack = Stack()
    max_stack = Stack()
    for command in reader:
        if command[0] == 'push':
            v = int(command[1])
            stack.push(v)
            if max_stack.is_empty() and v <= max_stack.head():
                max_stack.push(max_stack.head())
            else:
                max_stack.push(v)
        elif command[0] == 'pop':
            stack.pop()
            max_stack.pop()
        elif command[0] == 'max':
            print(max_stack.head())


if __name__ == "__main__":
    main()

In [ ]:
# http://e-maxx.ru/algo/stacks_for_minima#4
# Решение с помощью модифицированной очереди
import sys
from collections import deque


class ModQueue:
    def __init__(self):
        self.queue = deque([])

    def get_max(self):
        if self.queue:
            return self.queue[0]

    def push_back(self, i):
        while self.queue and self.queue[-1] < i:
            self.queue.pop()
        self.queue.append(i)

    def pop_front(self, removed_element):
        if self.queue and self.queue[0] == removed_element:
            self.queue.popleft()


def maxInWindow(n, A, m):
    queue = ModQueue()
    for k in range(m):
        queue.push_back(A[k])
    yield queue.get_max()
    for i in range(m, n):
        queue.push_back(A[i])
        queue.pop_front(A[i - m])
        yield queue.get_max()


def main():
    reader = (map(int, line.split()) for line in sys.stdin)
    [n], A, [m] = next(reader), list(next(reader)), next(reader)
    print(*maxInWindow(n, A, m))


if __name__ == "__main__":
    main()

In [ ]:
# http://e-maxx.ru/algo/stacks_for_minima#4
# Решение с помощью двух стеков
import sys
from collections import deque


class DoubleStack:
    def __init__(self):
        self.s1 = []
        self.s2 = []

    def add_element(self, new_elem):
        if not self.s1:
            self.s1.append((new_elem, new_elem))
        else:
            self.s1.append((new_elem, max(new_elem, self.s1[-1][1])))

    def pop_element(self):
        if not self.s2:
            while self.s1:
                elem, _ = self.s1.pop()
                maximum = elem if not self.s2 else max(elem, self.s2[-1][1])
                self.s2.append((elem, maximum))
        self.s2.pop()

    def get_max(self):
        if self.s1 and self.s2:
            return max(self.s1[-1][1], self.s2[-1][1])
        else:
            return self.s2[-1][1] if not self.s1 else self.s1[-1][1]


def maxInWindow(n, A, m):
    queue = DoubleStack()
    for k in range(m):
        queue.add_element(A[k])
    yield queue.get_max()
    for i in range(m, n):
        queue.add_element(A[i])
        queue.pop_element()
        yield queue.get_max()


def main():
    reader = (map(int, line.split()) for line in sys.stdin)
    [n], A, [m] = next(reader), list(next(reader)), next(reader)
    print(*maxInWindow(n, A, m))


if __name__ == "__main__":
    main()